# Exercícios do Minicurso

## Quadro-resumo dos exercícios

| Notebook               | Título do Exercício                                    | Objetivo Didático                                                                 |
|------------------------|--------------------------------------------------------|-----------------------------------------------------------------------------------|
| `basico.ipynb`         | Municípios prioritários para políticas públicas        | Filtrar e ordenar municípios com base em renda e número de famílias               |
| `joins.ipynb`          | Infos Municípios prioritários para políticas públicas  | Adicionar as informações do município                                             |
| `with_clause.ipynb`    | Mapeamento de famílias vulneráveis por município       | Cruzar tabelas e agregar famílias por município e perfil de instrução e renda     |
| `window_functions.ipynb` | Desigualdade de renda dentro das regiões             | Usar funções de janela para classificar as regiões por desilgualdade              |

---

## Detalhamento dos exercícios

### `basico.ipynb` – Municípios prioritários para políticas públicas

Liste os **10 municípios** onde:
- a **renda média familiar** é inferior a R$ 200, e  
- há mais de 50 famílias cadastradas.

Mostre:
- `cd_ibge`, `renda_media`, `total_familias`  
Ordene do menor para o maior valor médio de renda.

---

### `joins.ipynb` – Infos Municípios prioritários para políticas públicas

Liste os **10 municípios** onde:
- a **renda média familiar** é inferior a R$ 200, e  
- há mais de 50 famílias cadastradas.
- utilize a tabela de informações municípios do IBGE

Mostre:
- `municipio`, `uf`, `regiao`, `total_familias_em_risco`  
Ordene do menor para o maior valor médio de renda.

---

### `with_clause.ipynb` – Mapeamento de famílias vulneráveis por município

Liste os **10 municípios** com o maior número de famílias que:
- possuem **renda média familiar inferior a R$ 200**, e  
- têm **ao menos uma pessoa sem instrução formal** (`cod_curso_frequentou_pessoa_memb`).

Mostre:
- `municipio`, `uf`, `regiao`, `total_familias`  
Ordene pelo maior número de famílias em risco.

---

### `window_functions.ipynb` – Desigualdade de renda dentro das regiões

Calcule a média de renda das famílias por região
- 10% que recebem menos
- 10% que recebem mais
- diferença entre as médias

Mostre:
- `regiao`, `b_10`, `t_10`, `desigualdade`  
Ordene pela maior desigualdade.


# Resolução

In [ ]:
from common import DatabaseConnection

conn = DatabaseConnection("database.duckdb")

In [ ]:
conn.execute("""
    SELECT
        *
    FROM familias
    LIMIT 10;
""")

In [ ]:
conn.execute("""
    SELECT
        *
    FROM pessoas
    LIMIT 10;
""")

In [ ]:
conn.execute("""
    SELECT
        *
    FROM ibge_municipios
    LIMIT 10;
""")

# 1 - Municípios prioritários para políticas públicas

In [ ]:
conn.execute("""
    SELECT
        cd_ibge,
        AVG(vlr_renda_media_fam) AS renda_media,
        COUNT(*) AS total_familias
    FROM familias
    GROUP BY cd_ibge
    HAVING total_familias > 50 and renda_media < 200
    ORDER BY renda_media
    LIMIT 10;
""")

# 2 - Informações dos municípios prioritários para políticas públicas

In [ ]:
conn.execute("""
    SELECT
        m."municipio-nome" AS municipio,
        m."UF-sigla" AS uf,
        m."regiao-nome" AS regiao,
        AVG(f.vlr_renda_media_fam) AS renda_media,
        COUNT(f.*) AS total_familias
    FROM familias as f
    JOIN ibge_municipios as m ON f.cd_ibge = m."municipio-id"
    GROUP BY m."municipio-nome", m."UF-sigla", m."regiao-nome"
    HAVING total_familias > 50 and renda_media < 200
    ORDER BY renda_media
    LIMIT 10;
""")

# 3 - Mapeamento de famílias vulneráveis por município

In [ ]:
conn.execute("""
    SELECT
        id_familia,
        id_pessoa,
        CASE
            WHEN cod_curso_frequentou_pessoa_memb = 15 THEN 0
            WHEN cod_curso_frequentou_pessoa_memb is NULL THEN 0
            ELSE 1
        END AS frequentou_curso,
    FROM pessoas
    LIMIT 10;
""")

In [ ]:
conn.execute("""
    WITH frequentou_curso AS (
        SELECT
            id_familia,
            id_pessoa,
            CASE
                WHEN cod_curso_frequentou_pessoa_memb = 15 THEN 0
                WHEN cod_curso_frequentou_pessoa_memb is NULL THEN 0
                ELSE 1
            END AS frequentou_curso,
        FROM pessoas
    )
    SELECT
        m."municipio-nome" AS municipio,
        m."UF-sigla" AS uf,
        m."regiao-nome" AS regiao,
        COUNT(f.*) AS total_familias
    FROM familias AS f
    JOIN frequentou_curso AS fc ON f.id_familia = fc.id_familia
    JOIN ibge_municipios as m ON f.cd_ibge = m."municipio-id"
    WHERE fc.frequentou_curso = 0
    GROUP BY ALL
    ORDER BY total_familias DESC
    LIMIT 10;
""")

# 4 - Desigualdade de renda entre as regiões

In [ ]:
conn.execute("""
    SELECT
        f.vlr_renda_media_fam AS renda,
        m."regiao-nome" AS regiao,
        NTILE(10) OVER (PARTITION BY regiao ORDER BY renda) AS decil
    FROM familias f
    JOIN ibge_municipios m ON f.cd_ibge = m."municipio-id"
    WHERE f.vlr_renda_media_fam IS NOT NULL
    ORDER BY decil
""")

In [ ]:
conn.execute("""
    WITH renda_decil AS (
        SELECT
            f.vlr_renda_media_fam AS renda,
            m."regiao-nome" AS regiao,
            NTILE(10) OVER (PARTITION BY regiao ORDER BY renda) AS decil
        FROM familias f
        JOIN ibge_municipios m ON f.cd_ibge = m."municipio-id"
        WHERE f.vlr_renda_media_fam IS NOT NULL
    ), media_decil AS (
        SELECT
            regiao,
            AVG(CASE WHEN decil = 1 THEN renda END) AS b_10,
            AVG(CASE WHEN decil = 10 THEN renda END) AS t_10
        FROM renda_decil
        GROUP BY regiao
    )
    SELECT
        regiao,
        ROUND(b_10, 2) AS b_10,
        ROUND(t_10, 2) AS t_10,
        ROUND(t_10 - b_10, 2) AS desigualdade
    FROM media_decil
    ORDER BY desigualdade DESC;
""")

# Criando Gráficos

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

CLASSES = 10

desigualdade_df = conn.execute("""
    WITH renda_classe AS (
        SELECT
            f.vlr_renda_media_fam AS renda,
            m."regiao-nome" AS regiao,
            NTILE({classes}) OVER (PARTITION BY regiao ORDER BY renda) AS classe
        FROM familias f
        JOIN ibge_municipios m ON f.cd_ibge = m."municipio-id"
        WHERE f.vlr_renda_media_fam IS NOT NULL
    ), media_classe AS (
        SELECT
            regiao,
            AVG(CASE WHEN classe = 1 THEN renda END) AS b_{classes},
            AVG(CASE WHEN classe = {classes} THEN renda END) AS t_{classes}
        FROM renda_classe
        GROUP BY regiao
    )
    SELECT
        regiao,
        ROUND(b_{classes}, 2) AS b_{classes},
        ROUND(t_{classes}, 2) AS t_{classes},
        ROUND(t_{classes} - b_{classes}, 2) AS desigualdade
    FROM media_classe
    ORDER BY desigualdade DESC;
""".format(classes=CLASSES))

display(desigualdade_df)

plt.figure(figsize=(12, 6))
sns.barplot(x='regiao', y='desigualdade', data=desigualdade_df)
plt.title(f'Desigualdade de Renda por Região ({CLASSES} Classes)')
plt.xlabel('Região')
plt.ylabel('Desigualdade (Renda do Topo - Renda de Baixo)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Salvando Arquivos

In [ ]:
CLASSES = 10

conn.execute(f"""
    COPY (
        WITH renda_classe AS (
            SELECT
                f.vlr_renda_media_fam AS renda,
                m."regiao-nome" AS regiao,
                NTILE({CLASSES}) OVER (PARTITION BY regiao ORDER BY renda) AS classe
            FROM familias f
            JOIN ibge_municipios m ON f.cd_ibge = m."municipio-id"
            WHERE f.vlr_renda_media_fam IS NOT NULL
        ), media_classe AS (
            SELECT
                regiao,
                AVG(CASE WHEN classe = 1 THEN renda END) AS b_{CLASSES},
                AVG(CASE WHEN classe = {CLASSES} THEN renda END) AS t_{CLASSES}
            FROM renda_classe
            GROUP BY regiao
        )
        SELECT
            regiao,
            ROUND(b_{CLASSES}, 2) AS b_{CLASSES},
            ROUND(t_{CLASSES}, 2) AS t_{CLASSES},
            ROUND(t_{CLASSES} - b_{CLASSES}, 2) AS desigualdade
        FROM media_classe
        ORDER BY desigualdade DESC
    ) TO 'desigualdade_renda_regiao.csv' WITH (FORMAT CSV, HEADER);
""")